In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

def get_finger_states(hand_landmarks):
    fingers = [
        mp_hands.HandLandmark.THUMB_TIP,
        mp_hands.HandLandmark.INDEX_FINGER_TIP,
        mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
        mp_hands.HandLandmark.RING_FINGER_TIP,
        mp_hands.HandLandmark.PINKY_TIP
    ]
    finger_states = []
    for finger in fingers:
        if hand_landmarks.landmark[finger].y < hand_landmarks.landmark[finger - 2].y:
            finger_states.append(1)
        else:
            finger_states.append(0)
    return finger_states

def get_hand_gesture(hand_landmarks):
    finger_states = get_finger_states(hand_landmarks)
    
    # 엄지 위치 확인 (엄지척 제스처용)
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    thumb_ip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP]
    
    if finger_states == [1, 0, 0, 0, 0] and thumb_tip.x < thumb_ip.x:
        return "Thumbs up" # 엄지만 펴고, 엄지 끝이 손바닥 쪽으로 향함
    elif finger_states == [0, 1, 1, 0, 0]:
        return "V Sign" # 검지와 중지만 펴짐
    elif finger_states == [0, 1, 1, 1, 0]:
        return "Three" # 검지, 중지, 약지가 펴짐
    elif finger_states == [0, 1, 1, 1, 1]:
        return "Four" # 엄지를 제외한 네 손가락이 펴짐
    elif finger_states == [1, 1, 1, 1, 1]:
        return "pom" # 모든 손가락이 펴짐
    elif finger_states == [0, 1, 0, 0, 0]:
        return "index finger" # 검지만 펴짐
    elif finger_states == [1, 1, 0, 0, 1]:
        return "Rock" # 엄지, 검지, 새끼손가락만 펴짐
    else:
        return "기타 제스처"

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("웹캠을 찾을 수 없습니다.")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            gesture = get_hand_gesture(hand_landmarks)
            cv2.putText(image, f"Gesture: {gesture}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

# MediaPipe 손 랜드마크 설명

MediaPipe의 `mp_hands.HandLandmark` 열거형(enum)은 손의 주요 부위를 나타내는 21개의 랜드마크 포인트 정의.

1. `THUMB_TIP`: 엄지손가락 끝
2. `INDEX_FINGER_TIP`: 검지손가락 끝
3. `MIDDLE_FINGER_TIP`: 중지손가락 끝
4. `RING_FINGER_TIP`: 약지손가락 끝
5. `PINKY_TIP`: 새끼손가락 끝

각 랜드마크는 3D 공간에서의 x, y, z 좌표를 가집니다:
- x: 이미지의 가로 방향 위치 (0.0 - 1.0)
- y: 이미지의 세로 방향 위치 (0.0 - 1.0)
- z: 랜드마크의 깊이, 손목으로부터의 거리

주요 랜드마크 포인트:
- WRIST (0)
- THUMB_CMC (1), THUMB_MCP (2), THUMB_IP (3), THUMB_TIP (4)
- INDEX_FINGER_MCP (5), INDEX_FINGER_PIP (6), INDEX_FINGER_DIP (7), INDEX_FINGER_TIP (8)
- MIDDLE_FINGER_MCP (9), MIDDLE_FINGER_PIP (10), MIDDLE_FINGER_DIP (11), MIDDLE_FINGER_TIP (12)
- RING_FINGER_MCP (13), RING_FINGER_PIP (14), RING_FINGER_DIP (15), RING_FINGER_TIP (16)
- PINKY_MCP (17), PINKY_PIP (18), PINKY_DIP (19), PINKY_TIP (20)

참고:
- MCP: Metacarpophalangeal Joint (손가락 밑마디 관절)
- PIP: Proximal Interphalangeal Joint (손가락 첫째 관절)
- DIP: Distal Interphalangeal Joint (손가락 둘째 관절)
- IP: Interphalangeal Joint (엄지손가락 관절)